Ce TP présente le package python **graph-tool** qui permet de charger ou créer des graphes ainsi que de réaliser des mesures sur celui-ci. 

La plupart des commandes sont disponibles dans la cheat sheet fournie avec ce TP. Si vous souhaitez utiliser d'autres commandes disponibles, vous pouvez utiliser la documentation de graph-tool. 

# Partie A : Créer un graphe et le visualiser 

**1/ Importer graph_tool à l'aide de la commande `from graph_tool.all import *`** 

On aura aussi besoin du package random. 

In [1]:
from graph_tool.all import *
import random

**2/ Créer un graphe non orienté que l'on va appeler g**

In [2]:
g = Graph(directed = False)

Le graphe g est défini par le **dictionnaire d'interaction** suivant :  
{'A': ('B', 'E'), 
'B': ('A', 'C', 'D', 'E'), 
'C': ('D', 'E'), 
'D': ('B', 'C'), 
'E': ('A', 'B', 'C)}

**3/ Ajoutez les sommets et les arêtes à ce graphe pour représenter le graphe défini par ce dictionnaire.**

In [3]:
#Création de sommets
A = g.add_vertex()
B = g.add_vertex()
C = g.add_vertex()
D = g.add_vertex()
E = g.add_vertex()

# Creation d'arêtes
eAB = g.add_edge(A, B)
eBC = g.add_edge(B, C)
eCD = g.add_edge(C, D)
eBD = g.add_edge(B, D)
eBE = g.add_edge(B, E)
eCE = g.add_edge(C, E)
eAE = g.add_edge(A, E)

Pour avoir une représentation visuelle de votre graphe, vous pouvez utiliser la commande :

`graph_draw(g, vertex_text=g.vertex_index, output="le_nom_que_tu_veux.pdf")`

![image](graph.png)

# Partie B : Réaliser des mesures sur un graphe 

Le package graph-tool comporte plusieurs modules permettant de faire des calculs de valeurs sur les graphes, par exemple : 

- `graph-tool.util` pour accéder aux objets de nos graphes 
- `graph-tool.topology` pour accéder à des mesures topologiques sur nos graphes 
- `graph-tool.centrality` pour les mesures de centralité 
- `graph-tool.clustering` pour les mesures de clustering

Ici, nous allons utiliser uniquement ces modules même si d'autres existent. 

**1/ Trouver la liste des sommets et arêtes de ce graphe**

In [4]:
g.get_vertices()

array([0, 1, 2, 3, 4])

In [5]:
g.get_edges()

array([[0, 1],
       [0, 4],
       [1, 2],
       [1, 3],
       [1, 4],
       [2, 3],
       [2, 4]])

**2/ Trouver le nombre de voisins (donc le degré) de chaque sommet**

In [6]:
g.get_total_degrees([1])

array([4], dtype=uint64)

Nous nous intéressons maintenant à des mesures plus globales du graphe. 

Utilisez pour ça les fonctions `pseudo_diameter()`, `shortest_distance()`, `count_shortest_paths()` ou `all_shortest_paths()`.

**Attention** : `all_shortest_paths()` renvoie un itérateur

**3/ Indiquez le diamètre du graphe** 

In [7]:
pseudo_diameter(g)

(2.0,
 (<Vertex object with index '0' at 0x7fe09304f8d0>,
  <Vertex object with index '3' at 0x7fe09304f7b0>))

**3/ Indiquez la plus courte distance entre les sommets 0 et 2**

In [8]:
shortest_distance(g, source=0, target=2)

2

**4/ Indiquez le nombre et la liste des sommets du/des plus court(s) chemin(s) que vous trouverez entre les sommets 0 et 2**

In [9]:
count_shortest_paths(g, source=0, target=2)

2

In [10]:
for path in all_shortest_paths(g, source=0, target=2):
    print(path)

[0 4 2]
[0 1 2]


Nous allons maintenant nous intéresser aux propiétés du graphe grâce à la fonction `g.list_properties()` et nous allons y ajouter deux nouvelles propriétés, la betweenness et la closeness.

Pour ajouter une nouvelle propriété, il suffit d'utiliser la syntaxe suivante :

```python
g.vertex_properties["closeness"] = g.new_vertex_property("double")
# permet d'ajouter une propriété que l'on va appeler 
# "closeness" qui sera une nouvelle propriété sur les sommets 
# dont le type de valeur sera "double"

g.vertex_properties["closeness"] = closeness(g)
# closeness(g) renvoie un objet de type VertexPropertyMap 
# qui sera donc associée à la nouvelle propriété closeness de notre graphe.
```



Pour utiliser une propriété du graphe, pensez à y acceder en utilisant la syntaxe : `g.vp.nomdelapropriété[i]` ou `g.vertex_properties["nom_de_la_propriété"][i]` avec i l'index du sommet pour lequel on veut la propriété.

**ATTENTION** : `g.vp.nomdelapropriété[i]` ne fonctionne pas lorsque la propriété a un nom contenant un "_" 

**ATTENTION n°2** : pour la betweenness, il faudra aussi penser à modifier les propriétés des arêtes car betweenness(g) va renvoyer deux éléments : 

- `betweenness(g)[0]` correspond à la VertexPropertyMap 
- `betweenness(g)[1]` correspond à la EdgePropertyMap

**5/ Ajouter au graphe les propriétés correspondant à la closeness et la betweenness. Donner les valeurs de ces propriétés pour les sommets 0 à 2.**

In [11]:
g.vertex_properties["closeness"] = g.new_vertex_property("double") 
g.vertex_properties["closeness"] = closeness(g)

In [12]:
g.vertex_properties["betweenness"] = g.new_vertex_property("double")
g.edge_properties["betweenness"] = g.new_edge_property("double")
g.vertex_properties["betweenness"],g.edge_properties["betweenness"]  = betweenness(g)

In [14]:
for i in range(0, 3):
    print(f"Sommet n°{i}")
    print(f"Closeness : {g.vp.closeness[i]}")
    print(f"Betweenness : {g.vp.betweenness[i]}")

Sommet n°0
Closeness : 0.6666666666666666
Betweenness : 0.0
Sommet n°1
Closeness : 1.0
Betweenness : 0.3333333333333333
Sommet n°2
Closeness : 0.8
Betweenness : 0.08333333333333333


**5bis/ Que peut-on déduire de ces valeurs ?** 

**Sommet n°0** : 

- Betweenness de 0, il est donc rarement sur les chemins qui relie deux sommets du graphe. 
- Closeness de 0.6, il est tout de même proche des autres sommets, mais pas autant que les autres sommets, ce sont les sommets les plus excentrés du graphes. 

**Sommet n°1** : 

- Closeness de 1, il est trèèèèèèèèèèèèès proche des autres sommets du graphe. Il est relié par une arête à chacun d'entre eux. 
- Betweenness de 0.3, il est frequemment sur le chemin reliant 2 sommets. 

**Sommet n°2** : 

- Sommet intermédiaire au niveau des deux valeurs : assez proches des autres sommets et parfois sur leur chemin. 

**6/ On utilise la commande `graph_draw(g, vertex_fill_color=g.vp["closeness"], output = "colored_graph.pdf")` pour dessiner le graphe. Quelle est la particularité de ce dessin ?**

Les sommets sont coloriés en fonction de la valeur de leur closeness. 

![image](colored_graph.png)

# Partie C : Application sur un graphe biologique - A-t-on vraiment besoin d'ATP ? 

Le graphe **Reactome** est présent dans le dossier. Il est issu d'une base de donnée gratuite, open source de réactions biologiques (pathways). 

Nous allons maintenant le charger afin de reproduire les mesures effectuées sur le petit graphe, mais maintenant sur un graphe dirigé ayant un sens biologique. 

C'est un graphe dont les sommets sont de plusieurs types : 
'PhysicalEntity', 'ComplexAssembly', 'BiochemicalReaction', 'Protein', 'Complex', 'Dna', 'Catalysis', 'SmallMolecule', 'TemplateReaction', 'Control', 'Rna', 'TemplateReactionRegulation'.

Ainsi, si un sommet est de type 'Protein' ou 'Complex', ses voisins directs permettront de caractériser l'interaction avec ses voisins n+2. 

**L'objectif ici est d'utiliser les différentes fonctions de graph-tool pour extraire des informations sur la topologie du graphe et ses caractéristiques.**

**Une caractéristique des graphes biologiques sans échelle est qu'ils sont supposés robustes (tolérant aux pannes) mais fragiles (sensibles aux attaques ciblées).**


**Nous allons donc vérifier cela !** 

**1/ Charger le graphe présent dans le dossier.**

Ici le nom du fichier est : *PathwayCommons12.reactome.BIOPAX.graphml.gz*

In [16]:
g2 = load_graph("PathwayCommons12.reactome.BIOPAX.graphml.gz")

**2/ Donner le nombre de sommets et d'arêtes présents dans le graphe** 

In [17]:
print(f"Il y a {g2.num_vertices()} sommets dans le graphe.")
print(f"Il y a {g2.num_edges()} arêtes dans le graphe.")

Il y a 37976 sommets dans le graphe.
Il y a 67118 arêtes dans le graphe.


**3/ Lister les propriétés du graphe**

In [18]:
g2.list_properties()

_graphml_vertex_id     (vertex)  (type: string)
alias                  (vertex)  (type: string)
biopaxType             (vertex)  (type: string)
chebi                  (vertex)  (type: string)
color                  (vertex)  (type: string)
evidence               (vertex)  (type: string)
generic                (vertex)  (type: string)
hgnc                   (vertex)  (type: string)
input                  (vertex)  (type: int64_t)
name                   (vertex)  (type: string)
organism               (vertex)  (type: string)
provider               (vertex)  (type: string)
shape                  (vertex)  (type: string)
spaimCase              (vertex)  (type: string)
uniprot                (vertex)  (type: string)
uri                    (vertex)  (type: string)
_graphml_edge_id       (edge)    (type: string)
interaction            (edge)    (type: string)
spaim                  (edge)    (type: string)


**4/ On réalise cette commande. Que fait-elle ?**

In [19]:
len(find_vertex(g2, g2.vp.organism, "Homo sapiens"))

18835

Elle permet de donner le nombre de sommets qui ont comme propriété organisme "Homo sapiens". 

**5/ Trouver le/les sommet(s) avec le degré le plus élevé**

**Conseil** : chercher d'abord le degré maximal parmi tous les sommets puis chercher le/les sommets ayant un degré de cette valeur. 

**Rappel** : la commande `g2.get_total_degrees(range(0,g2.num_vertices()))` permet d'accéder à la liste de tous les degrés des sommets du graphe.

In [20]:
max_degree = max(g2.get_total_degrees(range(0,g2.num_vertices())))
print(max_degree)

2423


In [21]:
find_vertex(g2, "total", max_degree)

[<Vertex object with index '22346' at 0x7fe09591a1b0>]

Nous allons maintenant nous intéresser au sommet avec l'index **22346**. Ce sommet a donc un degré qui est maximal dans le graphe (2423). 

**6/ Quel est le terme utilisé en théorie des graphes pour décrire ce genre de sommets ?** 

C'est un HUB.

**7/ Trouver le type de sommet auquel correspond le sommet d'index 22346 (propriété : biopaxType) et son nom.**

In [22]:
g2.vp.biopaxType[22346]

'SmallMolecule'

In [23]:
g2.vp.name[22346]

'ATP'

Si vous ne l'avez pas trouvé, le sommet 22346 est une SmallMolecule et son nom est ... l'ATP ! Puisque son degré est de 2423, cela signifie qu'il y a 2423 interactions différentes entre l'ATP et d'autres composés. 

**8/ Calculer le demi-degré extérieur et intérieur du sommet représentant l'ATP. Qu'est ce que cela signifie dans le contexte de notre graphe ?**

In [24]:
g2.get_in_degrees([22346])

array([21], dtype=uint64)

In [25]:
g2.get_out_degrees([22346])

array([2402], dtype=uint64)

Cela signifie que l'ATP intervient dans 2423 réactions : 21 en tant que produit et 2402 en tant que réactif. 

On va s'intéresser maintenant à la plus large composante connexe du graphe. L'appartenance à celle-ci correspond à une propriété des sommets du graphe `label_largest_component()` (VertexProperty). 

**9/ Créer cette propriété pour le graphe.**

In [26]:
g2.vertex_properties["largest_component"] = g2.new_vertex_property("bool")
g2.vertex_properties["largest_component"] = label_largest_component(g2)

**10/ Est ce que l'ATP fait partie de cette composante connexe ?**

In [27]:
g2.vertex_properties["largest_component"][22346]

1

OUI ! 

On va donc s'intéresser à la composante connexe principale (la plus grosse, que l'on vient de définir). On peut l'extraire à l'aide d'une formule. 

**11/ Extraire la composante connexe la plus large et la stocker dans une variable g2_max_comp sous forme de Graph. Calculer le nombre de sommets dans ce graphe.**

In [28]:
g2_max_comp = extract_largest_component(g2, prune=True)
g2_max_comp.num_vertices()

9398

Vous devriez obtenir un nouveau graphe avec 9398 sommets. 

![image](comp_connexe.png)

**12/ Ajouter dans ce nouveau graphe g2_max_comp les propriétés décrivant la closeness, la betweenness et la PageRank de chaque sommet et donner ces valeurs pour le sommet représentant l'ATP.**

**ATTENTION** Dans ce nouveau Graph, l'ATP n'a plus le même index, il faut à nouveau le rechercher. Cette fois-ci, je vous aide : `find_vertex(g2_max_comp, g2_max_comp.vp.name, "ATP")`

In [29]:
find_vertex(g2_max_comp, g2_max_comp.vp.name, "ATP")

[<Vertex object with index '5541' at 0x7fb6f58d3030>]

In [30]:
g2_max_comp.vertex_properties["closeness"] = g2_max_comp.new_vertex_property("double")
g2_max_comp.vertex_properties["closeness"] = closeness(g2_max_comp)
g2_max_comp.vp.closeness[5541]

0.30729234793982996

In [31]:
g2_max_comp.vertex_properties["betweenness"] = g2_max_comp.new_vertex_property("double")
g2_max_comp.edge_properties["betweenness"] = g2_max_comp.new_edge_property("double")
g2_max_comp.vertex_properties["betweenness"],g2_max_comp.edge_properties["betweenness"]  = betweenness(g2_max_comp)
g2_max_comp.vp.betweenness[5541]

0.5815321339502544

In [32]:
g2_max_comp.vertex_properties["page_rank"] = g2_max_comp.new_vertex_property("double")
g2_max_comp.vertex_properties["page_rank"] = pagerank(g2_max_comp)
g2_max_comp.vp["page_rank"][5541]

0.018339966777905958

**12bis/ Qu'est ce que cela signifie concrètement pour l'ATP ?**

La closeness de l'ATP est de 0.3, ce n'est pas très élevé. Cela signifie que l'ATP n'est pas à une courte distance géodésique de tous les autres sommets du graphe. 
Par contre, la betweenness est assez élevé (0.58), cela signifie que l'ATP fait fréquemment partie des Plus Court Chemin (PCC) entre deux sommets quelconques du graphe. 

**13/ Calculer le diamètre du nouveau graphe g2_max_comp.**

In [33]:
pseudo_diameter(g2_max_comp)

(23.0,
 (<Vertex object with index '170' at 0x7fb6f58d3b70>,
  <Vertex object with index '9056' at 0x7fb6f58d3870>))

**14/ Le diamètre correspond au plus long des plus courts chemins du graphe. Ce chemin passe-t-il par le sommet ATP ?**

In [34]:
g2_max_comp.vertex(5541) in shortest_path(g2_max_comp, g2_max_comp.vertex(170), g2_max_comp.vertex(9056))[0]

True

graph-tool permet de modifier nos réseaux, d'ajouter ou de supprimer des sommets, des arêtes... Que se passe-t-il dans notre composante connexe si on supprime l'ATP ?

**15/ Supprimer le noeud correspondant à l'ATP dans le nouveau graphe g2_max_comp.**

**ATTENTION** Supprimer un noeud peut se faire de deux manières. Ici, n'oubliez pas de supprimer les ARETES reliant l'ATP à ces voisins. 



In [35]:
g2_max_comp.clear_vertex(5541)

# Autre méthode : 
# g2_max_comp.remove_vertex(5541) 
# Cela supprime le sommet mais cela réindexe tous les sommets. Ici, on va uniquement supprimer les arêtes 
# Pour éviter de perdre les index. 

**16/ Calculer la longueur du Plus Court Chemin entre les sommets 170 et 9056 dans le graphe g2_max_comp qui n'a plus d'ATP.**

In [36]:
len(shortest_path(g2_max_comp, g2_max_comp.vertex(170), g2_max_comp.vertex(9056))[0])

26

**17/ Qu'est ce que ces informations nous apportent sur le graphe et sur l'ATP ? Quel processus avons nous simulé ?**

L'ATP fait partie de nombreuses voies métaboliques au sein du corps humain. Au sein d'un graphe, il est ce que l'on peut appeler un HUB. On pourrait avoir peur qu'en cas de panne sur ce HUB, beaucoup de voies ne soient plus possible. Néanmoins, grâce à des voies de secours, certaines fonctions peuvent tout de même avoir lieu. 

Nous avons simulé une **panne**.

On va maintenant voir ce qui se passe en cas d'**attaque** sur l'ATP : c'est à dire un enchaînement de pannes sur les noeuds. Plusieurs possibilités en cas d'attaques : soit les hubs sont ciblés soit les sommets sont ciblés aléatoirement, soit d'autres types de sommets sont ciblés. 

Nous allons donc simuler les deux types d'attaques citées. 

On va donc itérer le processus :
- chercher le nouveau noeud avec le plus grand degré au sein du graphe g2_max_comp 
- supprimer les arêtes arrivant et partant de ce sommet 
- regarder si un chemin est toujours possible entre les sommets 170 et 9056
- s'arrêter quand ce n'est plus le cas. 

ou pour les sommets aléatoires : 
- supprimer aléatoirement les arêtes d'un des sommets du graphe.

Voici le code utilisé pour faire le premier type d'attaque et afficher le nombre de sommets à supprimer/isoler (puisqu'on supprime leurs arêtes mais on ne supprime pas le sommet) pour que les sommets 170 et 9056 ne soit plus reliés par aucun chemin : 

In [ ]:
nb_suppressed_hub = 1 # Car on a déjà supprimé ATP 
while shortest_path(g2_max_comp_copy, g2_max_comp_copy.vertex(170), g2_max_comp_copy.vertex(9056))[0] != []:
    max_degree = max(g2_max_comp_copy.get_total_degrees(range(0,g2_max_comp_copy.num_vertices())))
    hub = find_vertex(g2_max_comp_copy, "total", max_degree)
    g2_max_comp_copy.clear_vertex(hub[0]) # Hub est une liste avec un seul élément
    # mais la fonction clear_vertex prend en argument un élément et non une liste.
    nb_suppressed_hub +=1

print(nb_suppressed_hub)

**18/ Exécuter ce code et ensuite modifier le pour réaliser le second type d'attaque. Combien de sommet faut-il supprimer pour empêcher la réaction de s'éxécuter dans chaque attaque ? Cela coïncide-t-il avec les propriétés que vous connaissez sur les graphes sans échelle ?**

On vous conseille de faire cela sur des copies de g2_max_comp que vous pouvez créer en relançant le code `g2_max_comp_copy = extract_largest_component(g2, prune=True)`

In [38]:
g2_max_comp_copy = extract_largest_component(g2, prune=True)

nb_suppressed_hub = 1 # Car on a déjà supprimé ATP 
while shortest_path(g2_max_comp_copy, g2_max_comp_copy.vertex(170), g2_max_comp_copy.vertex(9056))[0] != []:
    max_degree = max(g2_max_comp_copy.get_total_degrees(range(0,g2_max_comp_copy.num_vertices()-1)))
    hub = find_vertex(g2_max_comp_copy, "total", max_degree)
    g2_max_comp_copy.clear_vertex(hub[0]) # Hub est une liste avec un seul élément
    # mais la fonction clear_vertex prend en argument un élément et non une liste.
    nb_suppressed_hub +=1

print(nb_suppressed_hub)

21


In [39]:
g2_max_comp_copy2 = extract_largest_component(g2, prune=True)

nb_suppressed_vertex = 1 # Car on a déjà supprimé ATP 
while shortest_path(g2_max_comp_copy2, g2_max_comp_copy2.vertex(170), g2_max_comp_copy2.vertex(9056))[0] != []:
    g2_max_comp_copy2.clear_vertex(random.randint(0, g2_max_comp_copy2.num_vertices()-1))
    nb_suppressed_vertex+=1

print(nb_suppressed_vertex)

666


On se retrouve avec une large différence : 
- si on cible les hubs, il faut uniquement 21 pannes successives pour que la réaction ne soit plus possible ;
- si on ne cible pas les hubs, la résistance est plutôt bonne et le nombre de pannes successives nécessaires pour que la réaction ne soit plus possible est beaucoup plus élevé (selon les tests entre 200 et 1400). 

Cela confirme la propriété des graphes sans échelle selon laquelle ces derniers sont robustes mais fragiles. 

**BONUS : Trouver les sommets qui ne sont plus atteignables dans le graphe à cause de la suppression de l'ATP**

**NB** : N'oubliez pas de vous replacez dans le graphe g2_max_comp.

In [40]:
## Il faut trouver les sommets qui ne sont plus dans la plus grande composante connexe du graphe. 
## Il faut donc créer la propriété "label_largest_component" dans le graphe g2_max_comp

g2_max_comp.vertex_properties["new_largest_component"] = g2_max_comp.new_vertex_property("bool")
g2_max_comp.vertex_properties["new_largest_component"] = label_largest_component(g2_max_comp)

In [41]:
unreachable_vertices = []
for i in range(0, g2_max_comp.num_vertices()-1):
    if g2_max_comp.vp["new_largest_component"][i] == 0:
        unreachable_vertices.append(g2_max_comp.vp.name[i])